In [2]:
import sys
sys.path.append('..')

import utils, selex_dca, indep_sites
import adabmDCA
import selex_distribution, energy_models, tree, data_loading, training, callback

import torch
from utils import one_hot
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
from adabmDCA.utils import get_device, get_dtype

/Users/scrotti/Aptamer2025py/selex_dca.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
experiment_id = "Dop8V030"
round_ids = ["ARN", "R01", "R02N"]

device = get_device("")
dtype = get_dtype("float64")

Running on CPU


In [4]:
sequences = [utils.sequences_from_file(experiment_id, round_id, device) for round_id in round_ids]

In [5]:
sequences_oh = [one_hot(seq).to(dtype=dtype, device=device) for seq in sequences]

In [6]:
# del sequences

In [7]:
fi0, _, _ = utils.frequences_from_sequences_oh(sequences_oh[0])

In [8]:
total_reads = torch.Tensor([s.shape[0] for s in sequences_oh])

In [29]:
import importlib
importlib.reload(callback)
importlib.reload(data_loading)
importlib.reload(training)

<module 'training' from '/Users/scrotti/Aptamer2025py/training.py'>

In [11]:
tr = tree.Tree()
tr.add_node(-1)
tr.add_node(0)

selected_modes = torch.BoolTensor([[1],[1]])

L, q = sequences_oh[0][0].shape
k = torch.zeros(L, q, dtype=dtype, device=device)
Ns0 = energy_models.IndepSites(k)

def mlp(n_hidden):
    return torch.nn.Sequential(
        torch.nn.Linear(L*q, n_hidden),
        torch.nn.ReLU(),
        torch.nn.Linear(n_hidden, 1)
    ).to(dtype=dtype, device=device)

nn = energy_models.GenericEnergyModel(mlp(2))

ps = selex_distribution.MultiModeDistribution(nn, normalized=False)
model = selex_distribution.MultiRoundDistribution(Ns0, ps, tr, selected_modes)

In [12]:
batch_size = 10**6
data_loaders = [data_loading.SelexRoundDataLoader(seq_oh, batch_size=batch_size) for seq_oh in sequences_oh]
n_rounds = len(data_loaders) 

In [13]:
n_chains = 10**3

chains = training.init_chains(n_rounds, n_chains, L, q, device, dtype)
log_weights = torch.zeros(n_rounds, n_chains, device=device, dtype=dtype)

In [14]:
callbacks = [callback.ConvergenceMetricsCallback()]

In [16]:
%load_ext line_profiler

In [ ]:
n_sweeps = 10
lr = 0.01
target_pearson = 1
max_epochs = 200

%lprun -f training.train training.train(model, data_loaders, total_reads, chains, n_sweeps, lr, max_epochs, target_pearson, callbacks=callbacks)

 0.00%[                              ] Epoch: 0/200 [00:00, ?it/s]

In [ ]:
callbacks[0].plot();